In [ ]:
import pygame
import numpy as np
import time


# Plateau initial
plateau = [
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0]
]

class Noeud:
    def __init__(self, plateau, joueur):
        self.plateau = plateau
        self.joueur = joueur
        self.phase_placement = True

    # 1) Méthode get_successeur modifiée
    def getSuccesseur(self):
        successeurs = []
        for x in range(3):
            for y in range(3):
                if self.plateau[x][y] == 0:  # Case vide où le joueur peut placer un 1
                    new_plateau = [row[:] for row in self.plateau]
                    new_plateau[x][y] = self.joueur  # Placement du pion sur la case vide

                    successeurs.append(Noeud(new_plateau, -self.joueur))  # Le joueur suivant

        return successeurs

    # 2) Méthode qui teste si un joueur a gagné
    def teste_gagne(self):
        indiceGagne = [
            [(0, 0), (0, 1), (0, 2)],
            [(1, 0), (1, 1), (1, 2)],
            [(2, 0), (2, 1), (2, 2)],
            [(0, 0), (1, 0), (2, 0)],
            [(0, 1), (1, 1), (2, 1)],
            [(0, 2), (1, 2), (2, 2)],
            [(0, 0), (1, 1), (2, 2)],
            [(0, 2), (1, 1), (2, 0)]
        ]

        for i in range(8):
            temp1 = self.plateau[indiceGagne[i][0][0]][indiceGagne[i][0][1]]
            temp2 = self.plateau[indiceGagne[i][1][0]][indiceGagne[i][1][1]]
            temp3 = self.plateau[indiceGagne[i][2][0]][indiceGagne[i][2][1]]

            if temp1 == temp2 == temp3:
                return temp1

        return 0

    # 3) Application Algorithme MiniMax
    def est_terminal(self):
        return self.teste_gagne() != 0 or all(self.plateau[x][y] != 0 for x in range(3) for y in range(3))

    def evaluation(self):
        gagnant = self.teste_gagne()
        if gagnant == self.joueur:
            return 10
        elif gagnant == -self.joueur:
            return -10
        return 0

    def minimax(self, profondeur, maximisant):
        if profondeur == 0 or self.est_terminal():
            return self.evaluation()

        if maximisant:
            max_eval = float('-inf')
            for successeur in self.getSuccesseur():
                eval = successeur.minimax(profondeur - 1, False)
                max_eval = max(max_eval, eval)
            return max_eval
        else:
            min_eval = float('inf')
            for successeur in self.getSuccesseur():
                eval = successeur.minimax(profondeur - 1, True)
                min_eval = min(min_eval, eval)
            return min_eval

    def meilleur_coup_minmax(self, profondeur):
        meilleur_valeur = float('-inf')
        meilleur_noeud = None

        for successeur in self.getSuccesseur():
            valeur = successeur.minimax(profondeur - 1, False)
            if valeur > meilleur_valeur:
                meilleur_valeur = valeur
                meilleur_noeud = successeur

        return meilleur_noeud

    def alphabeta(self, profondeur, alpha, beta, maximisant):
        if profondeur == 0 or self.est_terminal():
            return self.evaluation()

        if maximisant:
            max_eval = float('-inf')
            for successeur in self.getSuccesseur():
                eval = successeur.alphabeta(profondeur - 1, alpha, beta, False)
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break  # Pruning
            return max_eval
        else:
            min_eval = float('inf')
            for successeur in self.getSuccesseur():
                eval = successeur.alphabeta(profondeur - 1, alpha, beta, True)
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break  # Pruning
            return min_eval

    def meilleur_coup_alphabeta(self, profondeur):
        meilleur_valeur = float('-inf')
        meilleur_noeud = None
        alpha = float('-inf')
        beta = float('inf')

        for successeur in self.getSuccesseur():
            valeur = successeur.alphabeta(profondeur - 1, alpha, beta, False)
            if valeur > meilleur_valeur:
                meilleur_valeur = valeur
                meilleur_noeud = successeur

        return meilleur_noeud
    
    def ecraser(self, p):
        for x in range(3):
            for y in range(3):
                if self.plateau[x][y] == 0:
                    self.plateau[x][y] = p[x][y]

    def deplacements_possibles(self, x, y):
        deplacements = []
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < 3 and 0 <= ny < 3 and self.plateau[nx][ny] == 0:
                deplacements.append((nx, ny))
        return deplacements

    def ia_jouer(self):
        if self.phase_placement:
            placed = False
            for i in range(3):
                for j in range(3):
                    if self.plateau[i][j] == 0:
                        self.plateau[i][j] = -1
                        self.pions_ia += 1
                        placed = True
                        if self.pions_ia == 3:
                            self.phase_placement = False
                        self.joueur = 1
                        return
            if not placed and self.pions_ia < 3:
                self.joueur = 1
        else:
            for x in range(3):
                for y in range(3):
                    if self.plateau[x][y] == -1:
                        deplacements = self.deplacements_possibles(x, y)
                        if deplacements:
                            nx, ny = random.choice(deplacements)
                            self.plateau[x][y] = 0
                            self.plateau[nx][ny] = -1
                            self.joueur = 1
                            return

node = Noeud(plateau, 1)  # Joueur IA
profondeur = 3
maximisant = False
        
# Affichage de l'évaluation du plateau initial
#print("Évaluation du plateau initial : ", node.minimax(profondeur, maximisant))

# Trouver le meilleur coup pour le joueur
"""start_time = time.time()  # Début du chronomètre
meilleur_noeud_minimax = node.meilleur_coup_minmax(profondeur)
end_time = time.time()  # Fin du chronomètre
print("Meilleur coup MiniMax : ", meilleur_noeud_minimax.plateau)
print("Temps d'exécution MiniMax : {:.6f} secondes".format(end_time - start_time))
print("Meilleur coup pour le joueur minmax : ", meilleur_noeud_minimax.plateau)

start_time = time.time()  # Début du chronomètre
meilleur_noeud_alphabeta= node.meilleur_coup_alphabeta(profondeur)
end_time = time.time()  # Fin du chronomètre
print("Meilleur coup Alpha-Beta : ", meilleur_noeud_alphabeta.plateau)
print("Temps d'exécution Alpha-Beta : {:.6f} secondes".format(end_time - start_time))
print("Meilleur coup pour le joueur alphabeta : ", meilleur_noeud_alphabeta.plateau)

# Appliquer le meilleur coup
#node.ecraser(meilleur_noeud_minimax.plateau)
print("Plateau après application du meilleur coup : ", plateau)
"""


# Initialisation de pygame
pygame.init()

# Définition des dimensions de la fenêtre
WIDTH, HEIGHT = 400, 400
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Plateau de Fanorona 3x3")

# Couleurs
WHITE = (255, 255, 255)
GRAY = (128, 128, 128)
BLACK = (0, 0, 0)
LINE_COLOR = (0, 0, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
MODAL_BG = (50, 50, 50)  # Fond de la boîte modale
OVERLAY_COLOR = (0, 0, 0, 150)  # Fond semi-transparent
BUTTON_COLOR = (200, 0, 0)
BUTTON_HOVER_COLOR = (255, 0, 0)

# Paramètres du plateau
GRID_SIZE = 3
CELL_SIZE = WIDTH // GRID_SIZE

# Liste des nœuds
nodes = []
for row in range(GRID_SIZE):
    for col in range(GRID_SIZE):
        x = col * CELL_SIZE + CELL_SIZE // 2
        y = row * CELL_SIZE + CELL_SIZE // 2
        nodes.append(((x, y), (row, col)))

# Fonction pour dessiner le plateau et les lignes
def draw_board():
    pygame.draw.line(screen, LINE_COLOR, (65, 65), (WIDTH-65, 65), 3)
    pygame.draw.line(screen, LINE_COLOR, (65, 198), (WIDTH-65, 198), 3)
    pygame.draw.line(screen, LINE_COLOR, (65, 332), (WIDTH-65, 332), 3)
    pygame.draw.line(screen, LINE_COLOR, (65, 65), (65, 332), 3)
    pygame.draw.line(screen, LINE_COLOR, (198, 65), (198, 332), 3)
    pygame.draw.line(screen, LINE_COLOR, (332, 65), (332, 332), 3)
    pygame.draw.line(screen, LINE_COLOR, (65, 65), (WIDTH-65, 333), 3)
    pygame.draw.line(screen, LINE_COLOR, (62, 332), (WIDTH-65, 65), 3)

# Fonction pour dessiner les nœuds
def draw_nodes():
    for node, (row, col) in nodes:
        value = plateau[row][col]
        color = BLACK
        if value == 1:
            color = RED
        elif value == -1:
            color = BLUE
        pygame.draw.circle(screen, color, node, 10)

# Fonction pour afficher un message modal
def afficher_modal(screen, message):
    WIDTH, HEIGHT = screen.get_size()
    modal_width, modal_height = 300, 150
    modal_x, modal_y = (WIDTH - modal_width) // 2, (HEIGHT - modal_height) // 2

    # Création de la surface semi-transparente
    overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
    overlay.fill(OVERLAY_COLOR)

    # Création du modal
    modal_rect = pygame.Rect(modal_x, modal_y, modal_width, modal_height)

    # Bouton "Fermer"
    button_width, button_height = 80, 30
    button_x, button_y = modal_x + (modal_width - button_width) // 2, modal_y + modal_height - 50
    button_rect = pygame.Rect(button_x, button_y, button_width, button_height)

    font = pygame.font.Font(None, 28)
    text_surface = font.render(message, True, WHITE)
    text_rect = text_surface.get_rect(center=(modal_x + modal_width // 2, modal_y + modal_height // 3))

    button_font = pygame.font.Font(None, 22)
    button_text = button_font.render("Fermer", True, WHITE)
    button_text_rect = button_text.get_rect(center=(button_x + button_width // 2, button_y + button_height // 2))

    running = True
    while running:
        screen.blit(overlay, (0, 0))  # Appliquer le fond semi-transparent
        pygame.draw.rect(screen, MODAL_BG, modal_rect, border_radius=10)
        screen.blit(text_surface, text_rect)

        # Gestion du survol du bouton
        mouse_pos = pygame.mouse.get_pos()
        current_button_color = BUTTON_HOVER_COLOR if button_rect.collidepoint(mouse_pos) else BUTTON_COLOR
        pygame.draw.rect(screen, current_button_color, button_rect, border_radius=5)
        screen.blit(button_text, button_text_rect)

        pygame.display.flip()  # Mise à jour de l'écran

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                exit()
            if event.type == pygame.MOUSEBUTTONDOWN and button_rect.collidepoint(event.pos):
                running = False  # Ferme le modal quand on clique sur le bouton

# Boucle principale du jeu
def main():
    running = True
    compteur = 0
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                mouse_x, mouse_y = event.pos
                for node, index in nodes:
                    x, y = node
                    if (x - 10 <= mouse_x <= x + 10) and (y - 10 <= mouse_y <= y + 10):
                        print(f"Noeud cliqué: {index}")  
                        if compteur < 3 :                        
                            plateau[index[0]][index[1]] = -1                        
                            node = Noeud(plateau, 1)                        
                            meilleur_noeud_alphabeta= node.meilleur_coup_alphabeta(profondeur)                                                                        
                            node.ecraser(meilleur_noeud_alphabeta.plateau)                        
                            draw_nodes()                        
                            if node.teste_gagne() == 1 or node.teste_gagne() == -1:
                                afficher_modal(screen, node.test_gagne())
                                running = False
                            compteur += 1
                        
                            
        # Rafraîchir l'affichage
        screen.fill(GRAY)
        draw_board()
        draw_nodes()
        pygame.display.flip()

    pygame.quit()

# Lancer la fonction principale
if __name__ == "__main__":
    main()
